In [15]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/DACON/train_features.csv')
target = pd.read_csv('/content/drive/MyDrive/DACON/train_labels.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON/test_features.csv')
submission = pd.read_csv('/content/drive/MyDrive/DACON/sample_submission.csv')

In [16]:
import tensorflow as tf
from tensorflow import keras

In [19]:
train.iloc[:, 2:]

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...
1874995,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [20]:
X = tf.reshape(np.array(train.iloc[:, 2:]) , [-1,600,6])
print(X.shape)

(3125, 600, 6)


In [21]:
target

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [24]:
y = tf.keras.utils.to_categorical(target['label'])
print(y.shape)

(3125, 61)


In [99]:
n_timesteps, n_features, n_outputs = X.shape[1], X.shape[2], y.shape[1]

In [100]:
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.MaxPooling1D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [101]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 598, 64)           1216      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 596, 64)           12352     
_________________________________________________________________
dropout (Dropout)            (None, 596, 64)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 298, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 19072)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               1907300   
_________________________________________________________________
dense_9 (Dense)              (None, 61)               

In [102]:
model.fit(X,y, epochs= 20)

Epoch 1/20
98/98 [==============================] - 1s 6ms/step - loss: 20.1316 - accuracy: 0.2610
Epoch 2/20
98/98 [==============================] - 0s 5ms/step - loss: 2.7002 - accuracy: 0.4841
Epoch 3/20
98/98 [==============================] - 0s 5ms/step - loss: 2.2293 - accuracy: 0.5103
Epoch 4/20
98/98 [==============================] - 0s 5ms/step - loss: 1.8287 - accuracy: 0.5453
Epoch 5/20
98/98 [==============================] - 0s 5ms/step - loss: 1.2929 - accuracy: 0.6623
Epoch 6/20
98/98 [==============================] - 0s 5ms/step - loss: 0.9419 - accuracy: 0.7507
Epoch 7/20
98/98 [==============================] - 0s 5ms/step - loss: 0.7339 - accuracy: 0.8060
Epoch 8/20
98/98 [==============================] - 0s 5ms/step - loss: 0.5840 - accuracy: 0.8445
Epoch 9/20
98/98 [==============================] - 0s 5ms/step - loss: 0.3847 - accuracy: 0.8931
Epoch 10/20
98/98 [==============================] - 0s 5ms/step - loss: 0.3590 - accuracy: 0.9176
Epoch 11/20
98/98 

In [103]:
model.evaluate(X, y)

98/98 [==============================] - 0s 4ms/step - loss: 0.0378 - accuracy: 0.9946


[0.037780068814754486, 0.9945600032806396]

In [106]:
test_ = tf.reshape(np.array(test.iloc[:, 2:]) , [-1,600,6])

In [108]:
test_y = model.predict(test_)

In [109]:
test_y.shape

(782, 61)

In [110]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [111]:
submission.iloc[:,1:]=test_y

In [116]:
df = pd.DataFrame(submission)

In [117]:
df.to_csv('/content/drive/MyDrive/DACON/1dconv.csv')